In [2]:
import numpy as np

# Timing for the different functions of the code

## Hermite polynomials

Comparison of the recursive and scipy Hermite polynomials

In [10]:
from scipy import special

def hermite_pol_recursive(x, n):
    if n == 0:
        return 1
    elif n == 1:
        return 2*x
    else:
        return 2*x*hermite_pol_recursive(x, n-1) - 2*(n - 1)*hermite_pol_recursive(x, n-2)

def hermite_pol_scipy_1(x, n):
    H = special.hermite(n)
    return H(x)

def hermite_pol_scipy_2(x, n):
    return special.eval_hermite(n, x)

In [11]:
N = 10000
x = np.linspace(-10, 10, N)

print("Recursive Hermite polynomials")
%timeit hermite_pol_recursive(x, 1)
%timeit hermite_pol_recursive(x, 5)
%timeit hermite_pol_recursive(x, 10)

print("Scipy Hermite polynomials (option 1)")
%timeit hermite_pol_scipy_1(x, 1)
%timeit hermite_pol_scipy_1(x, 5)
%timeit hermite_pol_scipy_1(x, 10)

print("Scipy Hermite polynomials (option 2)")
%timeit hermite_pol_scipy_2(x, 1)
%timeit hermite_pol_scipy_2(x, 5)
%timeit hermite_pol_scipy_2(x, 10)

Recursive Hermite polynomials
6.65 µs ± 90.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
247 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
3.12 ms ± 17.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Scipy Hermite polynomials (option 1)
454 µs ± 4.68 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
543 µs ± 2.45 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
680 µs ± 2.45 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Scipy Hermite polynomials (option 2)
289 µs ± 2.07 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
330 µs ± 422 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
431 µs ± 648 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
